# 写
- write()方法必须接受字符串（str）类型的参数  
- writelines()方法将列表逐行写入
- 当参数是字符串的时候，writelines()方法等价于write()

```
# 以下3种写法等价，都是写入字符串“python is a cat”
with open('test.txt','w') as f:
    f.writelines(['python',' is',' a',' cat'])
    f.writelines('python is a cat')
    f.write('python is a cat')

# 以下2种写法等价，都是写入列表的字符串版本“['python',' is',' a',' cat']”
with open('test.txt','w') as f:
    f.write(str(['python',' is',' a',' cat']))
    f.writelines(str(['python',' is',' a',' cat']))

# 作为反例，以下写法都是错误的：
with open('test.txt','w') as f:
    f.writelines([2018,'is','a','cat']) # 含非字符串
    f.write(['python','is','a','cat']) # 非字符串
```
> 问题1：  
列表中就是有元素不是字符串，而且要把全部元素取出来，怎么办呢？  

答：  
```
In [37]: content=[1,' is',' everything']
In [38]: with open('test.txt','w') as f:
    ...:     for i in content:
    ...:         f.write(str(i) + '\n')
```

## 读
- file.read([size])  从文件读取指定字节数，没有给值或为负值读取所有
- file.readline([size])  读取整行，包括"\n"
- file.readlines([sizeint])  读取所有行并返回列表，若给定sizeint>0，则是设置一次读多少字节，这是为了减轻读取压力。  

$$readlines()用的较多，比较灵活，因为for循环是一种迭代器，每次加载部分内容，既减少内存压力，又方便逐行对数据处理$$

```
In [47]: with open('test.txt','r') as f:
    ...:     print(f.read())
1 is everything.
python is a cat.
this is the end.

In [48]: with open('test.txt','r') as f:
    ...:     print(f.readlines())
['1 is everything.\n', 'python is a cat.\n', 'this is the end.']

# 读取内容包含换行符，所以要strip()去掉换行符
In [62]: with open('test.txt','r') as f:
    ...:     for line in f.readlines():
    ...:         print(line.strip())
1 is everything.
python is a cat.
this is the end.
```

## 上下文管理器
- 它定义程序运行时需要建立的上下文，处理程序的进入和退出，实现了上下文管理协议，即在对象中定义了 `__enter__()` 和 `__exit__()` 方法。
> `__enter__()`：进入运行时的上下文，返回运行时上下文相关的对象，with 语句中会将这个返回值绑定到目标对象。   
`__exit__`(exception_type, exception_value, traceback)：退出运行时的上下文，定义在块执行（或终止）之后上下文管理器应该做什么。它可以处理异常、清理现场或者处理 with 块中语句执行完成之后需要处理的动作。  
上下文管理器必须同时提供 `__enter__()` 和 `__exit__()` 方法的定义，缺少任何一个都会导致 AttributeError。 

## 自定义上下文管理器

In [5]:
class OpenFile(object):
    def __init__(self,filename,mode):
        self.filename=filename
        self.mode=mode
    def __enter__(self):
        self.f=open(self.filename,self.mode)
        self.f.write("enter now\n")
        return self.f  #作为as说明符指定的变量的值
    def __exit__(self,type,value,tb):
        self.f.write("exit now")
        self.f.close()
        # 返回值等于 False，那么这个异常将被重新抛出到上层；返回值等于 True，那么这个异常就被忽略，继续执行后面的代码
        return False   #异常会被传递出上下文

## contextlib实现手感下文管理器 

contextmanager是要使用的装饰器，yield关键字将普通的函数变成了生成器。yield的返回值（ff）等于上例__enter__()的返回值，也就是as语句的值（f），而yield前后的内容，分别是_enter_() 和 _exit_() 方法里的内容。  

使用contextlib，可以避免类定义、`__enter__()` 和 `__exit()__` 方法，但是需要我们捕捉可能的异常（例如，yield只能返回一个值，否则会导致异常 RuntimeError），所以try…except语句不能忽略。

In [ ]:
from contextlib import contextmanager

@contextmanager
def open_file(name):
    ff = open(name, 'w')
    ff.write("enter now\n")
    try:
        yield ff
    except RuntimeError:
        pass
    ff.write("exit now")
    ff.close()

with open_file('test.txt') as f:
    f.write('Hello World!\n')